In [13]:
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')

### Preprocess encoding
[stackoverflow.com](https://stackoverflow.com/questions/26491448/how-to-fix-broken-utf-8-encoding-in-python)

In [ ]:
!python -m pip install ftfy

In [ ]:
from ftfy import fix_encoding

In [ ]:
str = "ÄÃ¢y lÃ  TrÆ°á»ng Äáº¡i há»c Kinh táº¿ TPHCM (UEH) náº±m trong Äáº¡i há»c Quá»‘c gia TPHCM; má»™t trong nhá»¯ng TrÆ°á»ng Äáº¡i há»c cÃ³ uy tÃ­n trong vÃ  ngoÃ i nÆ°á»›c; nÆ¡i Ä‘Ã o táº¡o ra nguá»“n nhÃ¢n lá»±c cháº¥t lÆ°á»£ng phá»¥c vá»¥ cho nhu cáº§u phÃ¡t triá»ƒn kinh táº¿ cá»§a Ä‘áº¥t nÆ°á»›c."

In [ ]:
fix_encoding(str)

'Đây là Trường Đại học Kinh tế TPHCM (UEH) nằm trong Đại học Quốc gia TPHCM; một trong những Trường Đại học có uy tín trong và ngoài nước; nơi đà o tạo ra nguồn nhân lực chất lượng phục vụ cho nhu cầu phát triển kinh tế của đất nước.'

### Combine files

In [ ]:
!python -m pip install XlsxWriter

In [ ]:
folder = r'U:\UEH kỳ 5\NLP\cuối kỳ - NLP'
inp_folder = folder + r'\data\raw-ggmap-crawl'
out_folder = folder + r'\data\combine'
out_filename = 'ueh-ggmap-reviewers.xlsx'
fileList = os.listdir(inp_folder)
fileList

['uehA.xlsx',
 'uehB.xlsx',
 'uehC.xlsx',
 'uehD.xlsx',
 'uehE.xlsx',
 'uehH.xlsx',
 'uehHOTEL.xlsx',
 'uehI.xlsx',
 'uehKTX135-1.xlsx',
 'uehKTX135-2.xlsx',
 'uehKTX4345.xlsx',
 'uehLibB.xlsx',
 'uehLibN.xlsx',
 'uehN.xlsx',
 'uehQ8.xlsx',
 'uehV.xlsx',
 'uehVL1.xlsx']

In [ ]:
dimensions = []
for file in os.listdir(inp_folder):
    if file.endswith('.xlsx'):
        file_path = os.path.join(inp_folder, file)
        df = pd.read_excel(file_path)
        
        # Get the dimensions (number of rows and columns)
        num_rows, num_cols = df.shape
        # Append dimensions to the list
        dimensions.append({'File': file, 'Rows': num_rows, 'Columns': num_cols})

dimensions_df = pd.DataFrame(dimensions)

total_rows = dimensions_df['Rows'].sum()
total_cols = dimensions_df['Columns'].sum()
total_row = {'File': 'Tổng', 'Rows': total_rows, 'Columns': total_cols}
dimensions_df = dimensions_df.append(total_row, ignore_index=True)

dimensions_df.style

,File,Rows,Columns
0,uehA.xlsx,1009,5
1,uehB.xlsx,668,5
2,uehC.xlsx,42,5
3,uehD.xlsx,31,5
4,uehE.xlsx,27,5
5,uehH.xlsx,11,5
6,uehHOTEL.xlsx,10,5
7,uehI.xlsx,37,5
8,uehKTX135-1.xlsx,111,5
9,uehKTX135-2.xlsx,3,5


In [ ]:
excelWriter = pd.ExcelWriter(out_folder + '/' + out_filename, engine='xlsxwriter')
files = [file.split('.',1)[0] for file in fileList]

for file in files:
    df = pd.read_excel(inp_folder + '/' + file + '.xlsx')
    df.to_excel(excelWriter, sheet_name=file, index=False)
excelWriter.save()

C:\Users\Admin\AppData\Local\Temp\ipykernel_18740\3973064587.py:7: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  excelWriter.save()


### Combine sheets

In [14]:
excel_file_path = r'combine\ueh-ggmap-reviewers-cleaned.xlsx'
xls = pd.ExcelFile(excel_file_path)
sheet_names = xls.sheet_names
sheet_names

['uehA',
 'uehB',
 'uehC',
 'uehD',
 'uehE',
 'uehH',
 'uehHOTEL',
 'uehI',
 'uehKTX135-1',
 'uehKTX135-2',
 'uehKTX4345',
 'uehLibB',
 'uehLibN',
 'uehN',
 'uehQ8',
 'uehV',
 'uehVL1']

In [15]:
combined_data = pd.DataFrame()

# Loop through each sheet
for sheet_name in sheet_names:
    # Read the data from the sheet
    df = pd.read_excel(xls, sheet_name)
    # Add a new column to identify the source sheet
    df['Cơ sở'] = sheet_name    
    # Concatenate the data to the combined_data DataFrame
    combined_data = pd.concat([combined_data, df], ignore_index=True)
    combined_data['Độ dài nội dung review'] = combined_data['context'].apply(lambda x: len(str(x)))
    combined_data = combined_data.sort_values(by='Độ dài nội dung review', ascending=False)
combined_data = combined_data.drop(columns=['Độ dài nội dung review'])

In [16]:
combined_data.to_excel('UEH-reviews.xlsx', index=False)